<a href="https://colab.research.google.com/github/koreamath/FinalProject-DeepLearning/blob/main/FinalProject_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy


import transformers
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from transformers import AdamW, get_linear_schedule_with_warmup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.5 MB/s eta 0:00:00


In [3]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


--2023-11-16 15:03:53--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  5.07MB/s    in 19s     

2023-11-16 15:04:12 (4.23 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2,     # The number of output labels. 2 for binary classification.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
import os
import random

# Function to read files from a directory
def read_files_in_directory(directory):
    files = os.listdir(directory)
    texts = []
    for file_name in files:
        with open(os.path.join(directory, file_name), 'r', encoding='utf8') as file:
            texts.append(file.read())
    return texts

# Load dataset
train_pos = read_files_in_directory('aclImdb/train/pos')
train_neg = read_files_in_directory('aclImdb/train/neg')
test_pos = read_files_in_directory('aclImdb/test/pos')
test_neg = read_files_in_directory('aclImdb/test/neg')

# Combine texts and labels
train_texts = train_pos + train_neg
train_labels = [1] * len(train_pos) + [0] * len(train_neg)  # 1 for positive, 0 for negative

# Optionally shuffle the data
combined = list(zip(train_texts, train_labels))
random.shuffle(combined)
train_texts, train_labels = zip(*combined)

# Now you can use train_test_split as you have done before
from sklearn.model_selection import train_test_split
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_texts, train_labels, random_state=2018, test_size=0.1)


In [7]:
from transformers import BertTokenizer
import torch

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize and encode sequences in the training set
train_encodings = tokenizer.batch_encode_plus(
    train_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=256,  # You can adjust this depending on your needs
    return_tensors='pt'
)

# Tokenize and encode sequences in the validation set
validation_encodings = tokenizer.batch_encode_plus(
    validation_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=256,  # Same as above
    return_tensors='pt'
)

# Extract inputs and masks from the encodings
train_inputs = train_encodings['input_ids']
train_masks = train_encodings['attention_mask']
validation_inputs = validation_encodings['input_ids']
validation_masks = validation_encodings['attention_mask']

# Convert labels to torch tensors
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)



In [8]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [9]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),
                  lr=2e-5,  # Learning rate
                  eps=1e-8  # Adam's epsilon for numerical stability
                 )
epochs = 4  # You can adjust this according to your needs
from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Number of warmup steps
                                            num_training_steps=total_steps)


In [11]:
import numpy as np
from sklearn.metrics import accuracy_score

# Function to calculate the accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
model = model.to('cuda')
# Training loop
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    model.train()

    total_loss = 0


    for step, batch in enumerate(train_dataloader):
        # Unpack the training batch from our dataloader.
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.to('cuda')
        b_input_mask = b_input_mask.to('cuda')
        b_labels = b_labels.to('cuda')

        # Clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss

        # Accumulate the training loss over all of the batches to calculate the average loss at the end.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate average loss over the training data
    avg_train_loss = total_loss / len(train_dataloader)

    # ========================================
    #               Validation
    # ========================================
    print("Running Validation...")

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0  # Initialize the number of evaluation steps

    for batch in validation_dataloader:

        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.to('cuda')
        b_input_mask = b_input_mask.to('cuda')
        b_labels = b_labels.to('cuda')

        # ... rest of your validation loop code ...

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

print("Training complete!")


======== Epoch 1 / 4 ========
Training...
Running Validation...
  Accuracy: 0.00
======== Epoch 2 / 4 ========
Training...
Running Validation...
  Accuracy: 0.00
======== Epoch 3 / 4 ========
Training...
Running Validation...
  Accuracy: 0.00
======== Epoch 4 / 4 ========
Training...
Running Validation...
  Accuracy: 0.00
Training complete!
